In [ ]:


import guidance
import re
import os

# set Openai API key
os.environ.setdefault("OPENAI_API_KEY", "sk-")

model = guidance.llms.OpenAI("gpt-3.5-turbo", chat_mode=True)


In [ ]:
# Define the chain of thought

chain_of_thought = guidance('''
{{#system~}}
You are a knowledgeable and efficient assistant.
{{~/system}}

{{#user~}}
Please analyze the following question:
{{query}}
Identify the exact topic and user intent in one sentence.
{{~/user}}

{{#assistant~}}
{{gen 'topic_intent' temperature=0 max_tokens=50}}
{{~/assistant}}

{{#user~}}
Based on the topic and intent, what information is required to adequately answer this question?
{{~/user}}

{{#assistant~}}
{{gen 'info_requirements' temperature=0 max_tokens=100}}
{{~/assistant}}

{{#user~}}
Here is a text source that might contain the answer:
{{knowledge}}
Analyze if this text contains the necessary information and respond with 'Yes' or 'No'.
{{~/user}}

{{#assistant~}}
{{gen 'text_analysis_conclusion' temperature=0 max_tokens=50}}
{{~/assistant}}

{{#if (contains text_analysis_conclusion 'Yes')}}
{{#user~}}
This is some knowledge:
{{knowledge}}
Generate a comprehensive answer based on the knowledge provided.
{{~/user}}                 
{{#assistant~}}
{{gen 'final_answer_based_on_knowledge' temperature=0 max_tokens=300}}
{{~/assistant}}

{{else}}
{{#user~}}

Generate a question that would elicit the missing information.
{{~/user}}
{{#assistant~}}                           
{{gen 'question_for_missing_info' temperature=0 max_tokens=300}}
{{~/assistant}}
{{/if}}
''', llm=model)


In [ ]:
# Define some questions and knowledge sources

questions = ["What are the health benefits of regular exercise?",
             "How do changes in ocean temperature affect marine life?",
             "What are the implications of AI in healthcare?"
]

knowledge = ["Regular exercise contributes to improved cardiovascular health, enhanced muscle strength, better mood regulation due to increased endorphin release, and aids in maintaining a healthy weight. It also reduces the risk of chronic diseases such as type 2 diabetes, heart disease, and certain types of cancer.",
            "The ocean is a vast and complex ecosystem. It provides habitat for countless species and plays a critical role in Earth's climate. Ocean currents are responsible for distributing heat around the globe, and the ocean absorbs large amounts of CO2 from the atmosphere.",
             "Artificial Intelligence (AI) is rapidly evolving and being integrated into various sectors, including healthcare. In healthcare, AI is used for data analysis, patient care, and diagnostic processes. While it offers efficiency and accuracy, concerns about data privacy and the ethical implications of AI in decision-making remain topics of discussion."
             ]


In [ ]:
# Inspect the output

for i, (question, knowledge) in enumerate(zip(questions, knowledge)):
    output = chain_of_thought(query=question, knowledge=knowledge)

    print(output)

    print(f'--------{i}----------')
    print("Question: ", question)
    print("Intent: " + output['topic_intent'])
    print("Info Requirements: " + output['info_requirements'])
    print("Analysis: " + output['text_analysis_conclusion'])
    
    # Check if the final answer is based on the knowledge or a generated question for missing info
    if 'final_answer_based_on_knowledge' in output:
        print("Final Answer: " + output['final_answer_based_on_knowledge'])
    elif 'question_for_missing_info' in output:
        print("Question for Missing Info: " + output['question_for_missing_info'])
    
    print('-------------------')

